# **Projet : Système de Recommandation en Temps Réel pour une Plateforme de Streaming avec Apache Spark et Hadoop**

## **Auteur :** Hamady GACKOU

## **Contexte** :

## Objectif :
Créer un pipeline de traitement de données distribuées pour une plateforme de streaming (films, séries, ou musique) en utilisant Hadoop et Apache Spark. L'objectif est d'exploiter des données massives (logs des utilisateurs, métadonnées des contenus, interactions) pour générer des recommandations en temps réel tout en optimisant le stockage et le traitement des données.



## **1. Architecture du Projet**
### **Stockage des Données :**
- HDFS : Stockage distribué des données des utilisateurs et des contenus.
- Parquet / Avro : Utilisation de formats de stockage efficaces pour les données structurées et semi-structurées.
- Hive / HBase : Base de données pour gérer l'accès aux informations des utilisateurs et des contenus.
- Kafka : Streaming en temps réel des événements utilisateur (clics, lectures, ajouts aux favoris).

### **Traitement des Données**
- Apache Spark (RDD, DataFrame, Spark SQL) :
- Nettoyage et transformation des données en RDD et DataFrames.
- Utilisation de Spark SQL pour requêter les données.
- Spark Streaming : Traitement en temps réel des logs utilisateur pour mise à jour dynamique des recommandations.
- Machine Learning avec Spark MLlib : Entraînement d'un modèle de recommandation basé sur ALS (Alternating Least Squares).
- Orchestration et Workflow
- Apache Airflow / Oozie : Gestion des workflows de traitement batch et en streaming.
## **2. Détails de Mise en Œuvre**
### **Phase 1 : Ingestion et Stockage des Données**
- Collecte des données

- Importation de logs utilisateur via Kafka.
- Importation des métadonnées des films via Sqoop (depuis une base relationnelle vers HDFS).
Stockage structuré

- Sauvegarde des données utilisateur en HBase.
- Sauvegarde des métadonnées en Hive sous format Parquet.
### **Phase 2 : Traitement et Analyse des Données**
- Nettoyage et Préparation des Données

- Utilisation de Spark RDD pour filtrer et normaliser les données utilisateur.
- Stockage des résultats intermédiaires sous Parquet.
- Analyse et Requêtage

- Création de vues SQL avec Spark SQL pour comprendre les comportements des utilisateurs.
- Identification des tendances et des contenus les plus populaires.
- Recommandation avec Spark MLlib

- Entraînement d’un modèle ALS (Collaborative Filtering) sur les interactions utilisateur.
- Optimisation des hyperparamètres et évaluation du modèle.
## **Phase 3 : Recommandations en Temps Réel**
- Traitement en Streaming avec Spark Streaming

- Capture des événements utilisateur (clics, visionnage) en temps réel via Kafka.
- Mise à jour dynamique du modèle de recommandation.
- Mise en Production du Modèle

- Stockage des recommandations mises à jour en HBase.
- API pour récupérer les recommandations utilisateur.
## **Phase 4 : Automatisation et Monitoring**
- Orchestration avec Apache Airflow / Oozie

- Planification des tâches batch (mise à jour des recommandations).
- Automatisation de l'entraînement du modèle tous les jours.
- Monitoring avec Spark UI et Prometheus

- Surveillance de la performance des jobs Spark.
- Optimisation des ressources sur YARN / Mesos.

📌 **Étape 1 : Configuration de l’environnement**
- Installez Java 8 et Apache Spark.
- Configurez les variables d'environnement nécessaires.
- Initialisez `findspark`.

In [ ]:
# Installer java  21,  Spark et ses dépendances
!apt-get install openjdk-21-jdk-headless -qq > /dev/null
! wget -q https://dlcdn.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
! tar xf /content/spark-3.5.4-bin-hadoop3.tgz

# Configuration des paths

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"

# Installer  findSpark
!pip install -q findspark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RecommenderSystem") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("Apache Spark est prêt ! 🚀")


**📌 Étape 2 : Ingestion et Stockage des Données**

Cette étape consiste à collecter les données utilisateur et les métadonnées des films, puis à les stocker dans HDFS et Hive.

**2.1 Télécharger un Jeu de Données (MovieLens)**

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o ml-latest-small.zip

**2.2 Charger les Données avec PySpark**

Créons un DataFrame PySpark pour charger les données :

In [ ]:
ratings = spark.read.csv("ml-latest-small/ratings.csv", header=True, inferSchema=True)
movies = spark.read.csv("ml-latest-small/movies.csv", header=True, inferSchema=True)


ratings.show(5)
movies.show(5)

# **📌 Étape 3 : Nettoyage et Préparation des Données**

Nous devons transformer et nettoyer les données pour les utiliser efficacement avec Spark MLlib.

**3.1 Supprimer les Valeurs Nulles et Filtrer les Données**

In [ ]:
ratings = ratings.dropna()
movies = movies.dropna()

# Vérifier les valeurs uniques
print(f"Nombre de films uniques : {movies.select('movieId').distinct().count()}")
print(f"Nombre d'utilisateurs uniques : {ratings.select('userId').distinct().count()}")


**3.2 Conversion des Données pour Spark MLlib**

Les algorithmes de recommandation utilisent des identifiants numériques. On va convertir les colonnes userId et movieId en integer.

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer_user = StringIndexer(inputCol="userId", outputCol="userIndex")
indexer_movie = StringIndexer(inputCol="movieId", outputCol="movieIndex")

ratings = indexer_user.fit(ratings).transform(ratings)
ratings = indexer_movie.fit(ratings).transform(ratings)

ratings.show(5)


## **📌 Étape 4 : Construction du Modèle de Recommandation**

Nous allons maintenant entraîner un modèle de filtrage collaboratif en utilisant ALS (Alternating Least Squares) dans Spark MLlib.

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Configuration du modèle ALS
als = ALS(
    userCol="userIndex",
    itemCol="movieIndex",
    ratingCol="rating",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop"
)


**4.2 Entraîner le Modèle**

Nous allons diviser les données en ensemble d’entraînement et de test (80% - 20%).

In [ ]:
train_data, test_data = ratings.randomSplit([0.8, 0.2], seed=42)

model = als.fit(train_data)

# Générer les prédictions
predictions = model.transform(test_data)
predictions.show(5)

## **📌 Étape 5 : Évaluation du Modèle**
Nous utilisons la Root Mean Square Error (RMSE) pour mesurer la précision des recommandations.

In [ ]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"RMSE du modèle : {rmse:.4f}")


## **📌 Étape 6 : Recommandations en Temps Réel avec Spark Streaming**

Nous allons maintenant simuler des événements utilisateur en temps réel avec Apache Kafka et Spark Streaming.

**6.1 Installer Apache Kafk**

In [ ]:
!pwd

In [ ]:
!wget -q https://downloads.apache.org/kafka/3.7.2/kafka-3.7.2-src.tgz
!tar -xzf kafka-3.7.2-src.tgz

In [ ]:
%cd /content/kafka-3.7.2-src
!chmod +x gradlew
# !./gradlew jar -PscalaVersion=2.13.12
!./gradlew  jar -PscalaVersion=2.13.12 -x test
#!./gradlew --no-daemon jar -PscalaVersion=2.13.12 -x test

In [ ]:
!./gradlew clean releaseTarGz -PscalaVersion=2.13.12

In [ ]:
!tar -xzf core/build/distributions/kafka_2.13-3.7.2.tgz

**6.2 Démarrer un Producteur Kafka (Simulation d'Événements)**

In [ ]:
%cd /content/kafka-3.7.2-src

In [ ]:
!./kafka_2.13-3.7.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.7.2/config/zookeeper.properties
!./kafka_2.13-3.7.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.7.2/config/server.properties


***Créer un topic Kafka nommé user-events :***

In [ ]:
!./kafka_2.13-3.7.2/bin/kafka-topics.sh --create --topic user-events --bootstrap-server localhost:9092 --partitions 1 --replication-factor 1

## **📌 Conclusion**
Nous avons mis en place un système de recommandation basé sur Spark et Hadoop, incluant :
- ✅ Ingestion des données avec HDFS et Kafka
- ✅ Traitement avec Spark SQL et Spark MLlib
- ✅ Génération des recommandations en temps réel avec Spark Streaming